In [55]:
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig, AutoModelWithLMHead
import numpy as np
from scipy.special import softmax
import seaborn as sns
import matplotlib.pyplot as plt
import re
import torch


In [56]:
reddit_comments_df=pd.read_csv('Datasets/reddit_comments_before_release.csv')
reddit_comments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260781 entries, 0 to 260780
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   body              260781 non-null  object
 1   post_date         260781 non-null  object
 2   upvotes           260781 non-null  int64 
 3   parent_id         194354 non-null  object
 4   top_level_id      260781 non-null  object
 5   post_title        260781 non-null  object
 6   post_id           260781 non-null  object
 7   post_create_date  260781 non-null  object
 8   movie_searched    260781 non-null  object
 9   release_date      260781 non-null  object
 10  date_diff         260781 non-null  int64 
dtypes: int64(2), object(9)
memory usage: 21.9+ MB


In [57]:
def remove_http_substrings(text):
    # Pattern to match substrings starting with 'http' followed by any characters until the next space
    pattern = r'http\S+'
    # Replace matched substrings with an empty string
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

reddit_comments_df['cleaned_body'] = reddit_comments_df['body'].apply(remove_http_substrings)

reddit_comments_df[['body','cleaned_body']].sample(5)

,body,cleaned_body
39257,That's when they show you the deleted scenes.,That's when they show you the deleted scenes.
216794,**Is there a** ***problem*** **gentlemen?**,**Is there a** ***problem*** **gentlemen?**
108583,"Is that Robert Smigel? Mark Ruffalo? Oh, it's ...","Is that Robert Smigel? Mark Ruffalo? Oh, it's ..."
6835,What the fuck was viper doing to her face?,What the fuck was viper doing to her face?
58157,Let me guess. A mean movie critic meets a sava...,Let me guess. A mean movie critic meets a sava...


In [58]:
def clean_text(text):
    # Replace non-alphanumeric characters (except for full stop) and newline characters with a space
    text_with_spaces = re.sub(r'[^\w\.]|\n', ' ', text)
    # Collapse multiple adjacent spaces into a single space
    cleaned_text = re.sub(r' +', ' ', text_with_spaces)
    return cleaned_text

In [36]:
reddit_comments_df['cleaned_body'] = reddit_comments_df['cleaned_body'].apply(clean_text)

reddit_comments_df[['body','cleaned_body']].sample(5)

,body,cleaned_body
27121,Where black sheep at?,Where black sheep at
123203,Who knows? \n\nI felt it from the first time I...,Who knows I felt it from the first time I saw ...
122137,No Franchise is safe,No Franchise is safe
220509,Would be one if A24 actually started an awards...,Would be one if A24 actually started an awards...
208945,He was in the second one. (though they probabl...,He was in the second one. though they probably...


In [59]:
#tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-imdb-sentiment")

#model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-imdb-sentiment")

In [8]:
def get_sentiment(text):
    input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')
    output = model.generate(input_ids=input_ids,max_length=2)
    dec = [tokenizer.decode(ids) for ids in output]
    #print(dec)
    label = dec[0]
    return label

In [11]:
#sample =get_sentiment("I dislike a lot that film")
#print(sample.split(' ')[1])

negative


In [10]:
#print(type(get_sentiment("I dislike a lot that film")))

<class 'str'>


In [ ]:
#reddit_comments_df['sentiment_using_t5_imdb'] = reddit_comments_df.apply(lambda x: get_sentiment(x['body']).split(' ')[1])

In [24]:
def process_batch(comments_batch):
    # Encode all comments in the batch
    input_ids = tokenizer.batch_encode_plus(comments_batch + ['</s>'], return_tensors='pt', padding=True, truncation=True, max_length=512)['input_ids']
    # Generate output for the entire batch
    output = model.generate(input_ids=input_ids, max_length=2)
    # Decode predictions
    labels = [tokenizer.decode(ids, skip_special_tokens=True) for ids in output[:-1]]
    return labels


In [15]:
batch_size = 1000  # Adjust based on your system's capability
num_batches = (len(reddit_comments_df) + batch_size - 1) // batch_size  # Calculate how many batches are needed

#num_batches

261

In [ ]:
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = start_idx + batch_size
    # Select the batch of comments
    comments_batch = reddit_comments_df.iloc[start_idx:end_idx]['body'].tolist()
    # Process the batch
    batch_labels = process_batch(comments_batch)
    # Store the results
    reddit_comments_df.loc[start_idx:end_idx-1, 'sentiment_using_t5_imdb'] = batch_labels
    # Optional: Save progress intermittently
    if i % 10 == 0:  # Save every 10 batches, adjust based on your preference
        reddit_comments_df.to_csv('partial_sentiment_analysis.csv', index=False)

In [38]:
process_batch(reddit_comments_df.iloc[0:10]['cleaned_body'].tolist())

['negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative']

In [37]:
#print(reddit_comments_df.iloc[0:10]['cleaned_body'].tolist())

['Damn even Scorsese pirated Yeezus.', ' Leo loves it though ', ' Not as much as Ethan Suplee ', 'Please put white people over this.', ' close up of dat face ', 'I can watch this allll day.', 'Is that Ethan Suplee in the background EDIT I think it may be ', ' awwww yeah it is ', 'Just watched the trailer went to this thread on this sub specifically to see if anyone had made a gif of that bit yet. You never disappoint me reddit', 'Is that Jon Bernthal in the background ']


In [40]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [60]:
MODEL2 = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer2 = AutoTokenizer.from_pretrained(MODEL2)
config2 = AutoConfig.from_pretrained(MODEL2)
model2 = AutoModelForSequenceClassification.from_pretrained(MODEL2)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [61]:

# PT

#model.save_pretrained(MODEL)
text = "Covid cases are increasing fast!"
text = preprocess(text)
print(text)
encoded_input = tokenizer2(text, return_tensors='pt')
output = model2(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores

Covid cases are increasing fast!


array([0.72357637, 0.22867927, 0.04774434], dtype=float32)

In [62]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
print(ranking)
print(scores.shape[0])
for i in range(scores.shape[0]):
    print(config2.id2label[ranking[i]])
    

[0 1 2]
3
negative
neutral
positive


In [63]:
config2.id2label

{0: 'negative', 1: 'neutral', 2: 'positive'}

In [66]:
def batch_sentiment_analysis(comments, batch_size=32):
    """
    Process comments in batches and return sentiment scores.

    Parameters:
    - comments (list of str): The comments to analyze.
    - batch_size (int): The size of batches to process at a time.

    Returns:
    - List of dicts with sentiment scores for each comment.
    """
    # Divide comments into batches
    batches = [comments[i:i + batch_size] for i in range(0, len(comments), batch_size)]
    sentiment_scores = []

    for batch in batches:
        # Encode batch
        encoded_input = tokenizer2(batch, padding=True, truncation=True, return_tensors='pt', max_length=512)
        # Get model output
        with torch.no_grad():
            output = model2(**encoded_input)
        # Apply softmax to get probabilities
        scores = softmax(output.logits.detach().numpy(), axis=1)
        
        # Store scores
        for score in scores:
            sentiment_scores.append({'negative': score[0], 'neutral': score[1], 'positive': score[2]})
    
    return sentiment_scores

In [65]:
comments = [
    "Covid cases are increasing fast!",
    "The new movie is fantastic!",
    "I'm not sure how I feel about the latest policy changes."
]
sentiment_scores = batch_sentiment_analysis(comments)
print(sentiment_scores)

[{'negative': 0.72357637, 'neutral': 0.22867927, 'positive': 0.047744345}, {'negative': 0.0034665263, 'neutral': 0.008983754, 'positive': 0.9875497}, {'negative': 0.6891944, 'neutral': 0.30030933, 'positive': 0.010496268}]


In [69]:
sentiment_df = pd.DataFrame()

In [77]:
import time

for start_idx in range(260000, 270000, 10000):  # Adjust the range as needed
    start_time = time.time()
    end_idx = start_idx + 10000
    # Select subset of rows for the current batch
    subset_comments = reddit_comments_df1['cleaned_body'].iloc[start_idx:end_idx].tolist()
    # Perform sentiment analysis on the current batch
    sentiment_scores = batch_sentiment_analysis(subset_comments)
    # Convert sentiment scores to DataFrame
    batch_df = pd.json_normalize(sentiment_scores)
    # Optional: Combine with original comments if needed
    batch_df['comments'] = subset_comments
    
    sentiment_df = pd.concat([sentiment_df, batch_df], ignore_index=True)
    # Save to CSV with a filename reflecting the index range
    filename = f"sentiment_scores_{start_idx}_{end_idx-1}.csv"
    batch_df.to_csv(filename, index=False)
    end_time = time.time()  # Capture end time
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"Batch {start_idx}-{end_idx-1} processed and saved to {filename}. Processing time: {elapsed_time:.2f} seconds.")

combined_df_filename = "combined_sentiment_scores_all1.csv"
sentiment_df.to_csv(combined_df_filename, index=False)
print(f"All batches combined and saved to {combined_df_filename}.")

Batch 260000-269999 processed and saved to sentiment_scores_260000_269999.csv. Processing time: 101.36 seconds.
All batches combined and saved to combined_sentiment_scores_all1.csv.


In [78]:
sentiment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260781 entries, 0 to 260780
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   negative  260781 non-null  float32
 1   neutral   260781 non-null  float32
 2   positive  260781 non-null  float32
 3   comments  260781 non-null  object 
dtypes: float32(3), object(1)
memory usage: 5.0+ MB


In [80]:
reddit_comments_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260781 entries, 0 to 260780
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   body              260781 non-null  object
 1   post_date         260781 non-null  object
 2   upvotes           260781 non-null  int64 
 3   parent_id         194354 non-null  object
 4   top_level_id      260781 non-null  object
 5   post_title        260781 non-null  object
 6   post_id           260781 non-null  object
 7   post_create_date  260781 non-null  object
 8   movie_searched    260781 non-null  object
 9   release_date      260781 non-null  object
 10  date_diff         260781 non-null  int64 
 11  cleaned_body      260781 non-null  object
dtypes: int64(2), object(10)
memory usage: 23.9+ MB


In [81]:
sentiment_merge = reddit_comments_df1.merge(sentiment_df, left_index=True, right_index=True)
sentiment_merge.sample(10)

,body,post_date,upvotes,parent_id,top_level_id,post_title,post_id,post_create_date,movie_searched,release_date,date_diff,cleaned_body,negative,neutral,positive,comments
36151,He sacrificed himself so the others could get ...,2014-05-08 16:36:07,2,chcplsg,chcplsg,New full-length trailer for 'Dawn of the Plane...,251d6x,2014-05-08 12:58:58,Dawn of the Planet of the Apes,2014-07-11,63,He sacrificed himself so the others could get ...,0.724227,0.257040,0.018733,He sacrificed himself so the others could get ...
205121,Well it certainly is a pretty grim tale,2019-09-04 19:23:44,2,eyzjetx,eyz1zhc,GRETEL & HANSEL Official Teaser Trailer (2020),czkm9i,2019-09-04 13:06:18,Gretel & Hansel,2020-01-31,148,Well it certainly is a pretty grim tale,0.901787,0.089228,0.008985,Well it certainly is a pretty grim tale
155729,[deleted],2018-06-09 14:14:35,5,e0dd7eu,e0cy5go,First Man (Official Trailer),8pp1f6,2018-06-09 00:31:14,First Man,2018-10-12,124,deleted,0.138816,0.801894,0.059289,deleted
127333,"It was a great ""haunted house"" type movie beca...",2016-12-25 06:54:43,542,dblutpc,dblso9k,Alien: Covenant - Official Trailer,5k6yfp,2016-12-25 05:00:11,Alien: Covenant,2017-05-19,144,It was a great haunted house type movie becaus...,0.434173,0.230303,0.335524,It was a great haunted house type movie becaus...
135099,I think he knew he could ride that money train...,2017-05-29 03:16:34,8,di66q5e,di5wmeh,LOGAN LUCKY | Official Trailer,6dwetc,2017-05-28 21:51:26,Logan Lucky,2017-08-18,80,I think he knew he could ride that money train...,0.064902,0.716132,0.218966,I think he knew he could ride that money train...
48022,;) you welcome.,2021-06-09 18:48:42,1,h16owxv,h15mva4,THE EYES OF TAMMY FAYE | Official Trailer | Se...,nvve49,2021-06-09 13:01:14,The Eyes of Tammy Faye,2021-09-17,99,you welcome.,0.022014,0.187035,0.790951,you welcome.
85770,"See the transition from joy to dread, he must ...",2015-12-31 19:13:26,16,cyhjw7y,cyh9pjs,'Zootopia' Official US Trailer #2,3ywslq,2015-12-31 13:45:01,Zootopia,2016-03-04,63,See the transition from joy to dread he must h...,0.529291,0.393967,0.076742,See the transition from joy to dread he must h...
137007,"No, it's more than that. The ghost's existenc...",2017-03-29 00:55:54,4,dfioy15,dfiefvh,A Ghost Story | Official Trailer HD | A24 (Cas...,61z6vm,2017-03-28 13:02:03,A Ghost Story,2017-07-07,99,No it s more than that. The ghost s existence...,0.257965,0.703821,0.038215,No it s more than that. The ghost s existence...
203611,Julie Kavner does Marge's voice on The Simpson...,2019-11-14 20:13:48,11,f7i7b9e,f7hmiw4,The SpongeBob Movie: Sponge on the Run (2020) ...,dw9wj6,2019-11-14 14:00:16,The SpongeBob Movie: Sponge on the Run,2021-03-04,475,Julie Kavner does Marge s voice on The Simpson...,0.019643,0.637103,0.343254,Julie Kavner does Marge s voice on The Simpson...
5978,oh gods please make more Incredibles movies,2012-06-20 11:58:20,17,c531s1f,c530zko,"Monsters inc. sequal, Monsters University trai...",vbimw,2012-06-20 07:44:34,Monsters University,2013-06-21,365,oh gods please make more Incredibles movies,0.029964,0.113917,0.856119,oh gods please make more Incredibles movies


In [83]:
sentiment_merge.drop(['comments'],axis=1,inplace=True)

In [85]:
filtered_df = sentiment_merge[~sentiment_merge['body'].str.contains('\[deleted\]|\[removed\]', na=False)]
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 252228 entries, 0 to 260780
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   body              252228 non-null  object 
 1   post_date         252228 non-null  object 
 2   upvotes           252228 non-null  int64  
 3   parent_id         187539 non-null  object 
 4   top_level_id      252228 non-null  object 
 5   post_title        252228 non-null  object 
 6   post_id           252228 non-null  object 
 7   post_create_date  252228 non-null  object 
 8   movie_searched    252228 non-null  object 
 9   release_date      252228 non-null  object 
 10  date_diff         252228 non-null  int64  
 11  cleaned_body      252228 non-null  object 
 12  negative          252228 non-null  float32
 13  neutral           252228 non-null  float32
 14  positive          252228 non-null  float32
dtypes: float32(3), int64(2), object(10)
memory usage: 27.9+ MB


In [86]:
filtered_df = filtered_df[filtered_df['cleaned_body'].str.strip() != '']
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 251097 entries, 0 to 260780
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   body              251097 non-null  object 
 1   post_date         251097 non-null  object 
 2   upvotes           251097 non-null  int64  
 3   parent_id         186502 non-null  object 
 4   top_level_id      251097 non-null  object 
 5   post_title        251097 non-null  object 
 6   post_id           251097 non-null  object 
 7   post_create_date  251097 non-null  object 
 8   movie_searched    251097 non-null  object 
 9   release_date      251097 non-null  object 
 10  date_diff         251097 non-null  int64  
 11  cleaned_body      251097 non-null  object 
 12  negative          251097 non-null  float32
 13  neutral           251097 non-null  float32
 14  positive          251097 non-null  float32
dtypes: float32(3), int64(2), object(10)
memory usage: 27.8+ MB


In [ ]:
filtered_df.to_csv("reddit_comments_sentiment_scores_hugging_face(twitter).csv",index=False)